The purpose of this notebook is to see how data from Volvo looks like

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from copy import deepcopy
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [4]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
pickles = {}
# for set_num in ['369']: #, '370', '371', '372', '375']:
pickles['data-369'] = pickle.load(open('/content/drive/My Drive/RISF/data-369.pickle', 'rb'))
print('369', ' loaded')
    
    # break point for not loading all data at once (unefficient) 
    # break

369  loaded


In [6]:
data = pickles['data-369']
data

,Timestamp,WetTankAirPressure,LongitudAcc,EngineSpeed,Fuel Rate,Engine Load,Boost Pressure,EngineAirInletPressure,AcceleratorPedalPos,VehicleSpeed,BrakePedalPos
0,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
1,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
2,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
3,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
4,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
10908295,8.299465e+10,11.65255,-0.2,596.500,4.081143,24.0,0.008618,102.0,0.0,2.331882,0.0
10908296,8.299465e+10,11.58360,-0.4,616.750,3.430526,19.5,0.000000,102.0,0.0,1.796760,11.2
10908297,8.299465e+10,11.51465,0.0,573.000,3.726261,20.5,0.000000,102.0,0.0,0.000000,18.4
10908298,8.299465e+10,11.44570,0.0,617.500,3.726261,20.0,0.000000,102.0,0.0,0.000000,0.0


In [7]:
VACT_EPOCH = datetime.datetime(year=2011,month=6,day=16,hour=5,minute=23,second=0)
def getDate(value):
    return VACT_EPOCH + datetime.timedelta(milliseconds=int(value))

data.Timestamp = data.Timestamp.apply(getDate)
data.Timestamp

0          2012-04-15 08:23:47.763
1          2012-04-15 08:23:48.457
2          2012-04-15 08:23:49.550
3          2012-04-15 08:23:50.715
4          2012-04-15 08:23:51.435
                     ...          
10908295   2014-01-31 19:27:06.824
10908296   2014-01-31 19:27:07.994
10908297   2014-01-31 19:27:08.672
10908298   2014-01-31 19:27:09.790
10908299   2014-01-31 19:27:10.884
Name: Timestamp, Length: 10908300, dtype: datetime64[ns]

Spliting data to train / test

In [8]:
data_train = data.loc[(data['Timestamp'] >= datetime.datetime(year=2012,month=5,day=20,hour=0,minute=0,second=1))
              & (data['Timestamp'] <= datetime.datetime(year=2012,month=7,day=20,hour=0,minute=0,second=1))]

X_train = np.array(data_train.drop(['Timestamp'], axis=1))
y_train = np.zeros(X_train.shape[0])

y_train[(data_train['Timestamp'] >= datetime.datetime(year=2012,month=6,day=20,hour=0,minute=0,second=1))
              & (data_train['Timestamp'] <= datetime.datetime(year=2012,month=7,day=20,hour=0,minute=0,second=1))] = 1

X_test = np.array(data.loc[(data['Timestamp'] >= datetime.datetime(year=2012,month=7,day=25,hour=0,minute=0,second=1))
              & (data['Timestamp'] <= datetime.datetime(year=2013,month=12,day=20,hour=0,minute=0,second=1))].drop(['Timestamp'], axis=1))
y_test = np.zeros(X_test.shape[0])

data_dir = {'X_train':X_train, 'y_train':y_train, 'X_test':X_test, 'y_test':y_test, 'name': 'bus-369'}

In [12]:
# !pip install pygod
# !pip install pyod
# !pip install typing
# Here we also need notebooks/utils.py from our repository
# in case of problems - make google colab use python ^3.8
# !wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
# !chmod +x mini.sh
# !bash ./mini.sh -b -f -p /usr/local
# !conda install -q -y jupyter
# !conda install -q -y google-colab -c conda-forge
# !python -m ipykernel install --name "py38" --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Requirement already up-to-date: typing in /usr/local/lib/python3.8/site-packages (3.7.4.3)


In [1]:
# import os
# os.kill(os.getpid(), 9)

Python 3.8.3


In [14]:
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '..')

from sklearn.metrics import roc_auc_score

from tqdm import tqdm
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from notebooks.utils import *

In [15]:
SEED = 23

In [16]:
clfs_names = ['ECOD', 'LOF', 'IForest', 'HBOS']
nan_datasets = []
results = {x: {} for x in clfs_names}

In [ ]:
# timer = Timer(timer_type="long_running")

In [17]:
from tqdm.contrib.concurrent import process_map

datasets_loop = tqdm([data_dir], desc="Datasets (outer loop)", position=0)
algorithms_loop = tqdm(clfs_names, desc=" Algorithms (inner loop)", position=1, leave=False)

 Algorithms (inner loop):   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
for dataset in datasets_loop:
    datasets_loop.set_description('BUS-369')
    for clf_name in algorithms_loop:
        algorithms_loop.set_description(clf_name)
        clf = new_clf(clf_name, SEED)
        # timer.start()
        clf.fit(dataset['X_train'])
        # timer.stop()
        
        # get the prediction labels and outlier scores of the training data
        y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
        y_train_scores = clf.decision_scores_  # raw outlier scores

        # get the prediction on the test data
        y_test_pred = clf.predict(dataset['X_test'])  # outlier labels (0 or 1)
        y_test_scores = clf.decision_function(dataset['X_test'])  # outlier scores
        print(y_test_pred)
        
        # if np.isnan(y_train_scores).any() or np.isnan(y_train_pred).any():
        #     nan_datasets.append(dataset['name'])
        #     results[clf_name][dataset['name']] = (np.nan, np.nan, np.nan, np.nan, np.nan) # AUC/ROC, Rank@N for train,test ; fit Time
        #     continue
        
        # roc_train=np.round(roc_auc_score(dataset['y_train'], y_train_scores), decimals=4)
        # prn_train=np.round(precision_n_scores(dataset['y_train'], y_train_scores), decimals=4)
        # roc_test=np.round(roc_auc_score(dataset['y_test'], y_test_scores), decimals=4)
        # prn_test=np.round(precision_n_scores(dataset['y_test'], y_test_scores), decimals=4)

        # results[clf_name]['BUS-369'] = (roc_train, prn_train, roc_test, prn_test)#, timer.time_sec)


[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 0]
[0 0 0 ... 0 0 0]
